# Implementation of SHA-1 Core on SoC Ported to Python

In [1]:
# Internal constants definitions
ADDR_I_CS_DATA      = 0x10
BITS_I_CS_DATA      = 1
ADDR_I_WE_DATA      = 0x18
BITS_I_WE_DATA      = 1
ADDR_I_ADDRESS_DATA = 0x20
BITS_I_ADDRESS_DATA = 8
ADDR_I_WRITE_DATA   = 0x28
BITS_I_WRITE_DATA   = 32
ADDR_O_READ_DATA    = 0x30
BITS_O_READ_DATA    = 32
ADDR_O_READ_CTRL    = 0x34
ADDR_O_ERROR_DATA   = 0x38
BITS_O_ERROR_DATA   = 1
ADDR_O_ERROR_CTRL   = 0x3c

In [2]:
# Load the overlay containing the IP
from pynq import Overlay

overlay = Overlay('/usr/local/lib/python3.6/dist-packages/sha1_pynq/bitstream/sha1_overlay.bit')

In [3]:
# See what attributes the overlay has
overlay?

In [4]:
# Access the the sha1 attribute to create a driver for the IP
sha1_ip = overlay.sha1_control_0
sha1_ip?

In [ ]:
def check_name():
    # Get the name and version of the device
    sha1_ip.write(ADDR_I_ADDRESS_DATA, 0)
    sha1_ip.write(ADDR_I_CS_DATA, 1)
    sha1_ip.write(ADDR_I_WE_DATA, 0)
    name0 = sha1_ip.read(ADDR_O_READ_DATA)
    print(bytearray.fromhex(hex(name0)[2:]).decode())
    
    sha1_ip.write(ADDR_I_ADDRESS_DATA, 1)
    sha1_ip.write(ADDR_I_CS_DATA, 1)
    sha1_ip.write(ADDR_I_WE_DATA, 0)
    name1 = sha1_ip.read(ADDR_O_READ_DATA)
    print(bytearray.fromhex(hex(name1)[2:]).decode())
    
    sha1_ip.write(ADDR_I_ADDRESS_DATA, 2)
    sha1_ip.write(ADDR_I_CS_DATA, 1)
    sha1_ip.write(ADDR_I_WE_DATA, 0)
    version = sha1_ip.read(ADDR_O_READ_DATA)
    print(bytearray.fromhex(hex(version)[2:]).decode())

In [ ]:
def write_block(block, ctrl_val):
    # Write the given block to the device
    mask = 0xffffffff000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
    
    sha1_ip.write(ADDR_I_CS_DATA, 1)
    sha1_ip.write(ADDR_I_WE_DATA, 1)
    write_word(0x10, (block & mask) >> 480)
    write_word(0x11, (block & (mask >> 32)) >> 448)
    write_word(0x12, (block & (mask >> 64)) >> 416)
    write_word(0x13, (block & (mask >> 96)) >> 384)
    write_word(0x14, (block & (mask >> 128)) >> 352)
    write_word(0x15, (block & (mask >> 160)) >> 320)
    write_word(0x16, (block & (mask >> 192)) >> 288)
    write_word(0x17, (block & (mask >> 224)) >> 256)
    write_word(0x18, (block & (mask >> 256)) >> 224)
    write_word(0x19, (block & (mask >> 288)) >> 192)
    write_word(0x1a, (block & (mask >> 320)) >> 160)
    write_word(0x1b, (block & (mask >> 352)) >> 128)
    write_word(0x1c, (block & (mask >> 384)) >> 96)
    write_word(0x1d, (block & (mask >> 416)) >> 64)
    write_word(0x1e, (block & (mask >> 448)) >> 32)
    write_word(0x1f, block & (mask >> 480))
    write_word(0x08, ctrl_val)
    sha1_ip.write(ADDR_I_WE_DATA, 0)
    sha1_ip.write(ADDR_I_CS_DATA, 0)
    
    sha1_ip.write(ADDR_I_CS_DATA, 1)
    

In [ ]:
def write_blocks(msg):
    import time
    # Convert the given message into a 512-bit padded block to be processed
    # Return the block as an integer
    from bitstring import BitArray
    binMsg = ''.join(format(ord(ch), '08b') for ch in msg)
    binMsg = "0b" + binMsg
    arr = BitArray(binMsg)
    msgSize = len(arr)
    arr.append('0b1')
    
    # pad the remaining bits and append the message size
    # this method ensures the padded message will have size that is multiple of 512
    while len(arr) % 512 != 448:
        arr.append('0b0')
    reserved = format(msgSize, '#066b')
    arr.append(reserved)
    
    # write each block in 512-bit chunks
    bitsWritten = 0
    #block1 = arr[bitsWritten:bitsWritten + 512].uint
    #block2 = arr[bitsWritten + 512:].uint
    while bitsWritten < len(arr):
        if bitsWritten == 0:
            # First block is written with ctrl_val = 0x1
            write_block(arr[bitsWritten:bitsWritten + 512].uint, 0x1)
            #write_block(block2, 0x1)
        else:
            # Next blocks are written with ctrl_val = 0x2
            write_block(arr[bitsWritten:bitsWritten + 512].uint, 0x2)
            #write_block(BLOCK2, 0x2)
        # Read the status and wait for it to be ready or valid
        #time.sleep(1)
        status = read_word(0x09)
        print("status:", status)
        while status == 0:
            status = read_word(0x09)
            print("status:", status)
        
        # Print the block that was sent and the digest it produced
        d = read_digest()
        print("block:", arr[bitsWritten:bitsWritten + 512])
        print("block digest:", hex(d)[2:])
        
        # Increment the number of bits written
        bitsWritten += 512

In [12]:
from IPython.core.debugger import set_trace

def read_word(address):
    '''
    Read a data word from the given address
    Return the data word that was obtained
    '''
    sha1_ip.write(ADDR_I_ADDRESS_DATA, address)
    sha1_ip.write(ADDR_I_CS_DATA, 1)
    sha1_ip.write(ADDR_I_WE_DATA, 0)
    word =0;
    while(sha1_ip.read(ADDR_O_READ_DATA) != 0):
        word = sha1_ip.read(ADDR_O_READ_DATA)
        break
    #sha1_ip.write(ADDR_I_CS_DATA, 0)
    return word

def read_digest():
    # Read the digest from the device
    # Return the digest data that was read
    digest_data_mask = 0xffffffff000000000000000000000000ffffffff
    mask = 0xffffffff00000000000000000000000000000000
    digest_data = (read_word(0x20) & digest_data_mask) << 128
    digest_data = digest_data | ((read_word(0x21) & digest_data_mask) << 96)
    digest_data = digest_data | ((read_word(0x22) & digest_data_mask) << 64)
    digest_data = digest_data | ((read_word(0x23) & digest_data_mask) << 32)
    digest_data = digest_data | read_word(0x24)
    return digest_data

def read_digest_test():
    digest = bytearray()
    for i in range(5):
        data = read_word(0x20 + i)
        digest.extend(data.to_bytes(4, byteorder='big'))
    return digest

def write_word(address, word):
    # Write the given word to the given address
    sha1_ip.write(ADDR_I_ADDRESS_DATA, address)
    sha1_ip.write(ADDR_I_WRITE_DATA, word)
    
def write_block_test(block, ctrl_val):
    sha1_ip.write(ADDR_I_CS_DATA, 1)
    sha1_ip.write(ADDR_I_WE_DATA, 1)
    #set_trace()
    j = 0
    for i in range(16):
        #write_word(0x10 + i, block[j:j + 4])
        word = int.from_bytes(block[j:j+4], byteorder='big')
        #write_word(0x10 + i, block[j:j + 4])
        write_word(0x10 + i, word)
        j += 4
    write_word(0x8, ctrl_val)
    sha1_ip.write(ADDR_I_WE_DATA, 0)
    sha1_ip.write(ADDR_I_CS_DATA, 0)
    sha1_ip.write(ADDR_I_CS_DATA, 1)

def write_blocks_test(msg):
    block = bytearray(msg, 'ascii')
    msgSize = len(msg)
    block.append(0x80)
    while len(block) % 64 != 56:
        block.append(0x00)
    reserved = (msgSize * 8).to_bytes(8, byteorder='big')
    block.extend(reserved)
    block = bytes(block)
    
    bytesWritten = 0
    while bytesWritten < len(block):
        if bytesWritten == 0:
            write_block_test(block[bytesWritten:bytesWritten + 64], 0x1)
            #write_word(0x8, 1)
        else:
            write_block_test(block[bytesWritten:bytesWritten + 64], 0x2)
            #write_word(0x8, 2)
        
        print("Block:", block[bytesWritten:bytesWritten + 64].hex())
        status = read_word(0x09)
        print("status:", status)
        while status == 0:
            status = read_word(0x09)
            print("status:", status)
        
        bytesWritten += 64

def hw_digest(msg):
    #write_blocks(msg)
    write_blocks_test(msg)
    #block = create_block(msg)
    #write_block(block)
    #write_word(0x08, 1)
    digest = read_digest_test()
    return digest

def sw_digest(msg):
    import hashlib
    digest = hashlib.sha1(msg.encode()).hexdigest()
    #digest = int(hashlib.sha1(msg.encode()).hexdigest(), 16)
    return digest

def test_sha1(msg):
    # Check if the message digests from the software and hardware are the same
    swDigest = sw_digest(msg)
    hwDigest = hw_digest(msg).hex()
    print("hw digest: {}".format(hwDigest))
    print("sw digest: {}".format(swDigest))
    
    if swDigest == hwDigest:
        print("Test Passed!")
    else:
        print("Test Failed!")
    
print("Single Block Test")
print("----------------------------------------")
test_sha1("abc")
print("----------------------------------------")
print("Double Block Test")
print("----------------------------------------")
test_sha1("abcdbcdecdefdefgefghfghighijhijkijkljklmklmnlmnomnopnopq")

Single Block Test
----------------------------------------
Block: 61626380000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000018
status: 3
hw digest: a9993e364706816aba3e25717850c26c9cd0d89d
sw digest: a9993e364706816aba3e25717850c26c9cd0d89d
Test Passed!
----------------------------------------
Double Block Test
----------------------------------------
Block: 6162636462636465636465666465666765666768666768696768696a68696a6b696a6b6c6a6b6c6d6b6c6d6e6c6d6e6f6d6e6f706e6f70718000000000000000
status: 3
Block: 000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001c0
status: 3
hw digest: fce65752d0f78ef035bd0b5ba8c8b678460655c8
sw digest: 84983e441c3bd26ebaae4aa1f95129e5e54670f1
Test Failed!


In [ ]:
def test_sha1(msg):
    # Check if the message digests from the software and hardware are the same
    swDigest = sw_digest(msg)
    hwDigest = hex(hw_digest(msg))[2:]
    print("hw digest: {}".format(hwDigest))
    print("sw digest: {}".format(swDigest))
    
    if swDigest == hwDigest:
        return "Test Passed!"
    else:
        return "Test Failed!"

In [ ]:
%pdb

In [ ]:
# Test 1
test_sha1("abc")

In [ ]:
# Test 2
test_sha1("Hello world")

In [ ]:
# Test 3
test_sha1("Reconfigurable GPU Computing Lab")

In [ ]:
# Custom message test
m = input("Enter a message to hash with SHA1: ")
test_sha1(m)

In [ ]:
import timeit
num = 100000
msg = input("Enter a message to hash with SHA1: ")
def sw_time():
    sw_digest(msg)
def hw_time():
    hw_digest(msg)
t1 = timeit.timeit('''msg = "Hello World!"
def sw_time():
    sw_digest(msg)''', number=num)
t2 = timeit.timeit('''msg = "Hello World!"
def hw_time():
    hw_digest(msg)''', number=num)
print("Time taken by software to run " + str(num) + " times = " + str(t1) + " seconds")
print("Time taken by hardware to run " + str(num) + " times = " + str(t2) + " seconds")